In [10]:
import pandas as pd
import os, glob, re

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import requests, json

In [3]:
onto = get_ontology("../pbn_t3_5_v0.4.owl").load()
comments = u.checkComments(onto)

ID: 0 	 Description: Creation of a knowledge graph based on a litterature review, augmented by use of LLMs.
ID: 1 	 Version: 0.4
ID: 2 	 Library: owlready2==0.45
ID: 3 	 Changes from: 0.3
ID: 4 	 Next: Check formulas for selecting most repeated items in groups
ID: 5 	 Changes: Adding new risks, groups of items, and new relationships
ID: 6 	 Creation: 22/01/2024
ID: 7 	 TODOs: Adding synonyms and solving classification with synonyms
ID: 8 	 VersionComment: Grouping of items added in 0.4
ID: 9 	 Project: PROBONO
ID: 10 	 Next: Linking benefits to mitigations groups
ID: 11 	 License: CC BY-NC-SA
ID: 12 	 Task: T3.5
ID: 13 	 Repository: https://github.com/mm80843/T3.5/
ID: 14 	 Author: Luc Jonveaux
ID: 15 	 Language: English


# Asking the BOK API.

This is the one in the current report, in the bok/data/main_api.py FastAPI app.

In [82]:
def askDef(term,k=10,temp=0.1,overwrite=False,seed=""):
    URL = "http://localhost:5000/ask/"
    #print(term)
    Q = "You are working on the topic of contagious diseases in smart and sustainable cities and neighbourhoods. In this context, what could be a definition or description of the following term: '"+term+"'?\nWrite two paragraphs and avoid bullet point lists."
    REQ = {
    "question": Q,
    "model": "gpt-3.5-turbo-1106",
    "temp": temp,
    "k": k,
    "overwrite": overwrite,
    "source": "local_risk_def",
    "seed": seed
    }

    H = h.hashme(REQ["question"]+str(REQ["seed"])+str(k)+str(temp))
    #print(H)
    cached = "cache/"+H+".md"
    #print(REQ)
    if not os.path.isfile(cached) or overwrite:
        x = requests.post(URL, json = REQ)
        #print(x.text)
        REFS = sorted(list(set(json.loads(x.text)["refs"])))
        answer = json.loads(x.text)["answer"]+"\n\nSources: "+str(REFS)
        h.svt(cached,answer)
    else:
        answer = h.ldt(cached)
    return answer

# Defining new properties

In [83]:
with onto:
    class has_Description(onto.PBNThing >> str):
        label = ["Short description"]
        pass
    class has_Note(onto.PBNThing >> str):
        label = ["Author note"]
        pass

# Getting the description

In [86]:
r = list(onto.RiskSubgroup.instances())
for rsg in r:
    rsg.has_Description = [askDef(rsg.label[0],k=15,temp=0.1,overwrite=True,seed="1")]
    #print(rsg.has_Description[0])

In [102]:
r = list(onto.TechSubgroup.instances())
for rsg in r:
    rsg.has_Description = [askDef(rsg.label[0],k=15,temp=0.1,overwrite=True,seed="1")]
    #print(rsg.has_Description[0])

In [87]:
r = list(onto.StakeholderSubgroup.instances())
for rsg in r:
    rsg.has_Description = [askDef(rsg.label[0],k=15,temp=0.1,overwrite=True,seed="1")]
    #print(rsg.has_Description[0])

# Removing old articles

In [95]:
onto.PBN__Article_310.label

['Rahim_impact_2022 rahim_impact_2022 nan rahim_impact_2022 nan nan rahim_impact_2022 nan nan nan rahim_impact_2022 nan nan nan nan rahim_impact_2022']

In [98]:
a =  list(onto.Article.instances())
gap_articles = [x for x in a if " nan " in x.label[0]]

In [99]:
for indiv in gap_articles :
    destroy_entity(indiv)

# Saving

In [101]:
onto.save("WIP.owl")